In [3]:
import pandas as pd
import sys
# HTTP request stuff
import json
import requests
import browser_cookie3

In [11]:
try:
    cookies = browser_cookie3.chrome(domain_name='ebay.com')
except PermissionError:
    print("Permission Error - Please close Chrome so I can access your browser cookies, you may reopen it after launching the script.", file=sys.stderr)
    exit(1)

cookie_dict = dict()
for i in cookies:
    cookie_dict[i.name] = i.value

In [69]:
url="https://www.ebay.com/sh/research/api/search?marketplace=EBAY-US&keywords=rtx+3070&dayRange=CUSTOM&endDate=1699680274511&startDate=1350000000000&categoryId=27386&conditionId=1000&minPrice=300&maxPrice=3000&offset=0&limit=50&tabName=SOLD&tz=America%2FLos_Angeles&modules=metricsTrends"
response2 = requests.get(url=url, cookies=cookie_dict)
response2.text
gpulist = ['gtx 960', 'gtx 970', 'gtx 980', 'gtx 980 ti', 'gtx 1060']

- GTX 960 - 980ti
- GTX 1060 - 1080ti + Titan Xp
- RTX 2060 - 2080: Super variants + Titan RTX
- RTX 3060 - 3090
- RTX 4060 - 4090
- Get some AMD GPU's
- 

In [76]:
def get_gpu_data(gpu_model: str, price_low: int, price_high: int):
    gpu_model = '+'.join(gpu_model.split(' '))
    url = f'https://www.ebay.com/sh/research/api/search?marketplace=EBAY-US&keywords={gpu_model}&dayRange=CUSTOM&endDate=1699680274511&startDate=1350000000000&categoryId=27386&conditionId=1000&minPrice={price_low}&maxPrice={price_high}&offset=0&limit=50&tabName=SOLD&tz=America%2FLos_Angeles&modules=metricsTrends'
    response = requests.get(url, cookies=cookie_dict)
    rlist = response.text.split('\n')
    requests_json = json.loads(rlist[2])
    # Prices data, Volume sold data
    return requests_json['series'][0]['data'], requests_json['series'][1]['data']


In [114]:
prices, vol = get_gpu_data('rtx 2070super', 0, 10000)

KeyError: 'data'

In [82]:
def get_cpu_data(cpu_model: str, price_low: int, price_high: int):
    cpu_model = '+'.join(cpu_model.split(' '))
    url = f'https://www.ebay.com/sh/research/api/search?marketplace=EBAY-US&keywords={cpu_model}&dayRange=CUSTOM&endDate=1699680274511&startDate=1350000000000&categoryId=164&conditionId=1000&minPrice={price_low}&maxPrice={price_high}&offset=0&limit=50&tabName=SOLD&tz=America%2FLos_Angeles&modules=metricsTrends'
    response = requests.get(url, cookies=cookie_dict)
    rlist = response.text.split('\n')
    requests_json = json.loads(rlist[2])
    return requests_json['series'][0]['data'], requests_json['series'][1]['data']


In [86]:
prices

[[1349679600000, 0, None],
 [1350284400000, 0, None],
 [1350889200000, 0, None],
 [1351494000000, 0, None],
 [1352102400000, 0, None],
 [1352707200000, 0, None],
 [1353312000000, 0, None],
 [1353916800000, 0, None],
 [1354521600000, 0, None],
 [1355126400000, 0, None],
 [1355731200000, 0, None],
 [1356336000000, 0, None],
 [1356940800000, 0, None],
 [1357545600000, 0, None],
 [1358150400000, 0, None],
 [1358755200000, 0, None],
 [1359360000000, 0, None],
 [1359964800000, 0, None],
 [1360569600000, 0, None],
 [1361174400000, 0, None],
 [1361779200000, 0, None],
 [1362384000000, 0, None],
 [1362985200000, 0, None],
 [1363590000000, 0, None],
 [1364194800000, 0, None],
 [1364799600000, 0, None],
 [1365404400000, 0, None],
 [1366009200000, 0, None],
 [1366614000000, 0, None],
 [1367218800000, 0, None],
 [1367823600000, 0, None],
 [1368428400000, 0, None],
 [1369033200000, 0, None],
 [1369638000000, 0, None],
 [1370242800000, 0, None],
 [1370847600000, 0, None],
 [1371452400000, 0, None],
 

In [121]:
nvidia_gpu_list = [('gtx 960', 80, 300), ('gtx 970', 80, 300), ('gtx 980', 130, 320), ('gtx 1060', 100, 300), ('gtx 1070', 120, 400), ('gtx 1080', 150, 700), ('gtx titan', 200, 1400), ('rtx 2060', 150, 800), ('rtx 2070', 150, 900), ('rtx 2080', 200, 1200), ('rtx titan', 300, 2200), ('rtx 3060', 200, 1200), ('rtx 3070', 250, 1600), ('rtx 3080', 400, 2300), ('rtx 3090', 500, 3200), ('rtx 4060', 300, 500), ('rtx 4070', 400, 1000), ('rtx 4080', 600, 1600), ('rtx 4090', 700, 3000)]

gpu_price_df = pd.DataFrame(columns=['timestamp', 'price', 'null', 'gpu_name'])
gpu_volum_df = pd.DataFrame(columns=['timestamp', 'volume', 'null', 'gpu_name'])
for gpu in nvidia_gpu_list:
    prices, vol = get_gpu_data(*gpu)

    tmp_df = pd.DataFrame(prices, columns=['timestamp', 'price', 'null'])
    tmp_df['gpu_name'] = len(tmp_df) * [gpu[0]]
    gpu_price_df = gpu_price_df.merge(tmp_df, how='outer')

    tmp_df2 = pd.DataFrame(vol, columns=['timestamp', 'volume', 'null'])
    tmp_df2['gpu_name'] = len(tmp_df2) * [gpu[0]]
    gpu_volum_df = gpu_volum_df.merge(tmp_df2, how='outer')

gpu_price_df.to_csv('./nvidia_prices.csv', index=False)
gpu_volum_df.to_csv('./nvidia_volume.csv', index=False)


In [120]:
gpu_price_df['gpu_name'].unique()

array(['gtx 960', 'gtx 970', 'gtx 980', 'gtx 980ti', 'gtx 1060',
       'gtx 1070', 'gtx 1070ti', 'gtx 1080', 'gtx 1080ti', 'gtx titan',
       'rtx 2060', 'rtx 2070', 'rtx 2080', 'rtx titan', 'rtx 3060',
       'rtx 3070', 'rtx 3080', 'rtx 3090', 'rtx 4060', 'rtx 4070',
       'rtx 4080', 'rtx 4090'], dtype=object)

In [107]:
gpu_price_df.head()

,timestamp,price,null,gpu_name
0,1349679600000,0.0,None,gtx 960
1,1350284400000,0.0,None,gtx 960
2,1350889200000,0.0,None,gtx 960
3,1351494000000,0.0,None,gtx 960
4,1352102400000,0.0,None,gtx 960


In [103]:
test_df = pd.DataFrame(columns=['timestamp', 'price', 'null', 'gpu_name'])
test_df2 = pd.DataFrame(prices, columns=['timestamp', 'price', 'null'])
test_df2['gpu_name'] = len(test_df2) * ['rtx 3070']
test_df.head()
test_df = test_df.merge(test_df2, how='outer')
test_df

,timestamp,price,null,gpu_name
0,1349679600000,0.00,None,rtx 3070
1,1350284400000,0.00,None,rtx 3070
2,1350889200000,0.00,None,rtx 3070
3,1351494000000,0.00,None,rtx 3070
4,1352102400000,0.00,None,rtx 3070
...,...,...,...,...
574,1696834800000,465.80,None,rtx 3070
575,1697439600000,421.87,None,rtx 3070
576,1698044400000,490.44,None,rtx 3070
577,1698649200000,429.27,None,rtx 3070


In [97]:
test_df2 = pd.DataFrame(prices, columns=['timestamp', 'price', 'null'])
test_df2

,timestamp,price,null
0,1349679600000,0.00,None
1,1350284400000,0.00,None
2,1350889200000,0.00,None
3,1351494000000,0.00,None
4,1352102400000,0.00,None
...,...,...,...
574,1696834800000,465.80,None
575,1697439600000,421.87,None
576,1698044400000,490.44,None
577,1698649200000,429.27,None
